In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/jiegouhua714/train.parquet
/kaggle/input/jiegouhua714/test.parquet
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/working/__notebook__.ipynb


In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import sys
import pickle
import glob
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
import random

random.seed(75)
from tqdm.notebook import tqdm_notebook
from functools import partial, reduce

### warnings setting
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

#### model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
import catboost
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import joblib
import pickle
from tqdm.notebook import tqdm_notebook
import uuid

##### LOGGING Stettings #####
import logging

# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# Create formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(name)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', )
handler.setFormatter(formatter)
# Set STDERR handler as the only handler
logger.handlers = [handler]

#### plots
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors

sns.set(rc={'axes.facecolor': '#f9ecec', 'figure.facecolor': '#f9ecec'})

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

### Plotly settings
theme_palette = {
    'base': '#a3d3eb',
    'complementary': '#ebbba3',
    'triadic': '#eba3d3',
    'backgound': '#f6fbfd'
}

temp = dict(layout=go.Layout(font=dict(family="Ubuntu", size=14),
                             height=600,
                             legend=dict(  #traceorder='reversed',
                                 orientation="v",
                                 y=1.15,
                                 x=0.9),
                             plot_bgcolor=theme_palette['backgound'],
                             paper_bgcolor=theme_palette['backgound']))

In [3]:
train = pd.read_parquet('/kaggle/input/jiegouhua714/train.parquet')


In [4]:
## Define some features by category
features = train.drop(['customer_ID', 'S_2'], axis=1).columns.to_list()
cat_vars = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'month',
            'day_of_week']
num_vars = list(filter(lambda x: x not in cat_vars, features))

# devide nums vars by AmEx Categories
delequincy_vars = filter(lambda x: x.startswith('D') and x not in cat_vars, features)
spend_vars = filter(lambda x: (x.startswith('S')) and (x not in cat_vars), features)
payment_vars = filter(lambda x: x.startswith('P') and x not in cat_vars, features)
balance_vars = filter(lambda x: x.startswith('B') and x not in cat_vars, features)
risk_vars = filter(lambda x: x.startswith('R') and x not in cat_vars, features)

with open('features.pkl', 'wb') as f:
    pickle.dump(features, f)

with open('cat_vars.pkl', 'wb') as f:
    pickle.dump(cat_vars, f)

with open('num_vars.pkl', 'wb') as f:
    pickle.dump(num_vars, f)

In [5]:
class BatchGenerator:
    def __init__(self, df, batch_feature='customer_ID', keep_features=features, n_batchs=750):
        self.df = df
        self.batch_feature = batch_feature
        self.keep_feature = list(set([batch_feature] + keep_features))
        self.n_batchs = n_batchs

    def __iter__(self):
        unique_vals = self.df[self.batch_feature].unique()
        batch_size = int(np.ceil(len(unique_vals) / self.n_batchs))
        groups = self.df.groupby(self.batch_feature).groups
        n_batchs = min(self.n_batchs, int(np.ceil(len(unique_vals) / batch_size)))
        for i in range(n_batchs):
            keys = unique_vals[i * batch_size:(i + 1) * batch_size]
            idx = [i for s in keys for i in groups[s]]
            if i == n_batchs - 1:
                keys = unique_vals[(i + 1) * batch_size:]
                idx = idx + [i for s in keys for i in groups[s]]
            yield self.df.loc[idx, self.keep_feature]


week_days = {1: 'Mon', 2: 'Tue', 3: 'Wen', 4: 'Thu', 5: 'Fri', 6: 'Sat', 7: 'Sun'}

In [6]:
def extract_date_vars(df, date_var='S_2', sort_by=['customer_ID', 'S_2'], week_days=week_days):
    # change to datetime
    df[date_var] = pd.to_datetime(df[date_var])
    # sort by custoner ther by date
    df = df.sort_values(by=sort_by)
    # extract some date characteristics
    # year has not a very
    # month
    df['month'] = df[date_var].dt.month
    # day of week
    df['day_of_week'] = df[date_var].apply(lambda x: x.isocalendar()[-1])
    return df


group_names = ["delequincy_vars", "spend_vars", "payment_vars", "balance_vars", "risk_vars"]


In [7]:
def row_rise_aggregation(df,
                         group_vars=[delequincy_vars, spend_vars, payment_vars, balance_vars, risk_vars],
                         group_names=group_names,
                         save=True):
    print('shape before row_rise_aggregation', df.shape)
    for group_name, group_var in zip(group_names, group_vars):
        df[group_name + '_sum'] = df[group_var].sum(axis=1)
        df[group_name + '_mean'] = df[group_var].mean(axis=1)
        df[group_name + '_missing'] = df.isnull().sum(axis=1)
    print('shape after row_rise_aggregation', df.shape)
    if save:
        df.reset_index(drop=False).to_feather(f"row_agg_{str(uuid.uuid4())}.ftr")
        return df['customer_ID'].nunique()
    return df

In [8]:
def column_rise_aggregation(df, num_vars=num_vars, cat_vars=cat_vars, save=True):
    print('shape before column_rise_aggregation', df.shape)
    group_names = filter(lambda x: '_vars' in x, df.columns)
    num_agg = df.groupby("customer_ID")[list(set(list(num_vars) + list(group_names)))].agg(
        ['mean', 'std', 'min', 'max', 'last'])
    num_agg.columns = ['_'.join(x) for x in num_agg.columns]

    cat_agg = df.groupby("customer_ID")[list(set(list(cat_vars) + ['month', 'day_of_week']))].agg(
        ['count', 'last', 'nunique', pd.Series.mode])
    cat_agg.columns = ['_'.join(x) for x in cat_agg.columns]

    mode_cols = filter(lambda x: x.endswith('_mode'), cat_agg.columns)
    for col in mode_cols:
        cat_agg[col] = cat_agg[col].apply(lambda x: random.choice(str(x).strip('[]').split()))
    #concat the two dataframes
    df = pd.concat([num_agg, cat_agg], axis=1)
    del num_agg, cat_agg

    gc.collect()
    print('shape after column_rise_aggregation', df.shape)
    if save:
        df.reset_index(drop=False).to_feather(f"col_agg_{str(uuid.uuid4())}.ftr")
        return len(df)  #df['customer_ID'].nunique()
    return df

In [9]:
# from https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977
def get_difference(df, num_features):
    res = []
    customer_ids = []
    for customer_id, df in tqdm_notebook(df.groupby(['customer_ID'])):
        # Get the differences
        diff_df = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        res.append(diff_df)
        customer_ids.append(customer_id)
    # Concatenate
    res = np.concatenate(res, axis=0)
    # Transform to dataframe
    res = pd.DataFrame(res, columns=[col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    res['customer_ID'] = customer_ids
    print('final shape', res.shape)
    #       df = df.merge(res, on='customer_ID', how='inner')
    #       df.reset_index(drop=False).to_feather(f"diff_{str(uuid.uuid4())}.ftr")
    return res  #df['customer_ID'].nunique()

In [10]:
def save_partition(df, prefix='train'):
    global c
    df.reset_index(drop=True).to_feather(f'{prefix}_{c}.ftr')
    c = c + 1
    return df['customer_ID'].nunique()

In [11]:
N_BATCHS = 100
c = 0

samples_df = BatchGenerator(train, batch_feature='customer_ID', keep_features=features + ['S_2'], n_batchs=N_BATCHS)
processed_elements = sum(map(partial(save_partition, prefix='train'), tqdm_notebook(samples_df, total=N_BATCHS)))

del train
gc.collect()

  0%|          | 0/100 [00:00<?, ?it/s]

18

In [12]:
processed = []
n_paths = 0
for path in tqdm_notebook(glob.glob('train_*.ftr')):
    # apply on train
    sample_df = pd.read_feather(path)
    diff_df = get_difference(sample_df, num_features=num_vars)
    sample_df = sample_df.merge(diff_df, on='customer_ID', how='inner')

    #   if sample_df.shape[1]<300:
    sample_df = extract_date_vars(sample_df)
    all_num_vars = num_vars + list(map(lambda x: x + '_diff1', num_vars))
    sample_df = row_rise_aggregation(sample_df, save=False)
    sample_df = column_rise_aggregation(sample_df, num_vars=all_num_vars, save=False)

    
    for col in tqdm_notebook(num_vars):
        try:
            sample_df[f'{col}_last_mean_diff'] = sample_df[f'{col}_last'] - sample_df[f'{col}_mean']
        except:
            pass

    sample_df = sample_df.reset_index()

    sample_df.reset_index(drop=True).to_feather(path)
    

train = pd.concat(map(lambda sample_df: pd.read_feather(sample_df), tqdm_notebook(glob.glob('train_*.ftr'))))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/4590 [00:00<?, ?it/s]

final shape (4590, 178)
shape before row_rise_aggregation (55372, 369)
shape after row_rise_aggregation (55372, 384)
shape before column_rise_aggregation (55372, 384)
shape after column_rise_aggregation (4590, 1893)


  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/4590 [00:00<?, ?it/s]

...

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/4590 [00:00<?, ?it/s]

final shape (4590, 178)
shape before row_rise_aggregation (55104, 369)
shape after row_rise_aggregation (55104, 384)
shape before column_rise_aggregation (55104, 384)
shape after column_rise_aggregation (4590, 1893)


  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
## Left join with labels:
labels = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')
print(labels.shape, labels['customer_ID'].nunique())
labels = labels.set_index('customer_ID')
train = train.set_index('customer_ID')
train['target'] = labels['target']
# del labels
gc.collect()
# save result
train.reset_index().to_feather('train_fin_df.ftr')

(458913, 2) 458913


In [14]:
N_BATCHS = 300
c=0

test = pd.read_parquet('/kaggle/input/jiegouhua714/test.parquet')
n_cid = test.customer_ID.nunique()
print(test.shape, n_cid )
samples_df = BatchGenerator(test, batch_feature='customer_ID', keep_features=features+['S_2'], n_batchs=N_BATCHS)
processed_elements = sum(map(partial(save_partition, prefix='test'), tqdm_notebook(samples_df, total=N_BATCHS)))
    
del test
gc.collect()

(11363762, 190) 924621


  0%|          | 0/300 [00:00<?, ?it/s]

18

In [15]:
processed = 0

      
for path in tqdm_notebook(glob.glob('test_*.ftr')):
    # apply on train
    sample_df = pd.read_feather(path)
    diff_df = get_difference(sample_df, num_features=num_vars)
    sample_df = sample_df.merge(diff_df, on='customer_ID', how='inner')
        
 #   if sample_df.shape[1]<300:
    sample_df = extract_date_vars(sample_df)
    all_num_vars = num_vars + list(map(lambda x: x+'_diff1', num_vars))
    sample_df = row_rise_aggregation(sample_df, save=False)
    sample_df = column_rise_aggregation(sample_df, num_vars=all_num_vars, save=False)
        
    print("diff between last and mean transaction")
    for col in tqdm_notebook(num_vars):
        try:
            sample_df[f'{col}_last_mean_diff'] = sample_df[f'{col}_last'] - sample_df[f'{col}_mean']
        except:
            pass

    sample_df = sample_df.reset_index()
        
    sample_df.reset_index(drop=True).to_feather(path)
    print("save processed", path)
        
    processed += sample_df.customer_ID.nunique()
        
test = pd.concat(map(lambda sample_df: pd.read_feather(sample_df), tqdm_notebook(glob.glob('test_*.ftr'))))
test.reset_index(drop=True).to_feather('test_fin_df.ftr')

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/2804 [00:00<?, ?it/s]

final shape (2804, 178)
shape before row_rise_aggregation (34462, 369)
shape after row_rise_aggregation (34462, 384)
shape before column_rise_aggregation (34462, 384)
shape after column_rise_aggregation (2804, 1893)
diff between last and mean transaction


  0%|          | 0/177 [00:00<?, ?it/s]

save processed test_299.ftr


...

  0%|          | 0/3083 [00:00<?, ?it/s]

final shape (3083, 178)
shape before row_rise_aggregation (38013, 369)
shape after row_rise_aggregation (38013, 384)
shape before column_rise_aggregation (38013, 384)
shape after column_rise_aggregation (3083, 1893)
diff between last and mean transaction


  0%|          | 0/177 [00:00<?, ?it/s]

save processed test_225.ftr


  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
test = pd.concat(map(lambda sample_df: pd.read_feather(sample_df), tqdm_notebook(glob.glob('test_*.ftr'))))


In [ ]:
test.info()
gc.collect()

In [ ]:
test.set_index('customer_ID')

In [ ]:
train

In [ ]:
test.reset_index(drop=True).to_feather('test_fin_df.ftr')

In [ ]:
train = pd.concat(map(lambda sample_df: pd.read_feather(sample_df), tqdm_notebook(glob.glob('train_*.ftr'))))

In [ ]:
train = pd.read_feather('train_fin_df.ftr')

In [ ]:
train

In [ ]:
def reduce_size(df):
# Transform float64 columns to float32
    print("reduce float data size")
    cols = list(df.dtypes[df.dtypes == 'float64'].index)
    for col in tqdm_notebook(cols):
        df[col] = df[col].astype(np.float32)
    # Transform int64 columns to int32
    print("reduce cat data size")
    cols = list(df.dtypes[df.dtypes == 'int64'].index)
    for col in tqdm_notebook(cols):
        df[col] = df[col].astype(np.int32)
    return df
        
train = reduce_size(train)
test = reduce_size(test)

In [ ]:
def missing_values_table(df):
    # Total missing values by column
    mis_val = df.isnull().sum()

    # Percentage of missing values by column
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # build a table with the thw columns
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

# Missing values for training data
missing_values_train = missing_values_table(test)
#cm = sns.color_palette('Set2', as_cmap=True)
#missing_values_train[:20]#.style.background_gradient(cmap=cm)
THRESHOLD = 80
print(test.shape)
drop_cols = missing_values_train[missing_values_train['% of Total Values']>THRESHOLD].index.to_list()
print(f"Drop {len(drop_cols)} features with more than {THRESHOLD}% of missing values")

test = test.drop(drop_cols,axis=1)
print("Training data shape after dropping highly missing values columns", test.shape)

In [ ]:
corr = train.corrwith(train['target'], axis=0)
corr = corr[corr.notna()].sort_values(key=abs, ascending=False)
THRESHOLD = 0.15
CORR_SELECTION=True
if CORR_SELECTION:
    selected_feats = corr[corr.abs()>THRESHOLD].index
    train = train[list(selected_feats)]
    print(f"Training data shape after dropping uncorrelated features"
          f"(threshold Pearson correlation = {THRESHOLD})", 
          train.shape)

In [ ]:
train[list(selected_feats)]

In [ ]:
gc.collect()

In [ ]:
NaN_Val = np.array(train.isnull().sum())
NaN_prec = np.array((train.isnull().sum() * 100 / len(train)).round(2))
NaN_Col = pd.DataFrame([np.array(list(train.columns)).T,NaN_Val.T,NaN_prec.T,np.array(list(train.dtypes)).T], index=['Features','Num of Missing values','Percentage','DataType']
).transpose()
pd.set_option('display.max_rows', None)
NaN_Col

In [ ]:
features = [col for col in train.columns if col != "customer_ID"]


In [ ]:
for col in features:
    train[col] = train[col].fillna(train[col].median())

In [ ]:
types = train.dtypes
target_col = 'target'

cat_cols = list(types[types.apply(lambda x:not(str(x).startswith('float')))].index)
cat_cols = list(filter(lambda x:x!=target_col, cat_cols))
features = list(train.drop(target_col, axis=1).columns)
gc.collect()
print('len cat_col', len(cat_cols))
print('len features', len(features))
    
with open('features.pkl', 'wb') as f:
    pickle.dump(features, f)

with open('cat_cols.pkl', 'wb') as f:
    pickle.dump(cat_cols, f)

In [ ]:
train['target'] = train["target"].astype('int')
print('Transform all String features to category.\n')
os.makedirs('label_encoders')

for usecol in tqdm_notebook(cat_cols):
#    print(usecol)
    train[usecol] = train[usecol].astype('str')
#    test[usecol] = test[usecol].astype('str')

    #Fit LabelEncoder
    le = LabelEncoder().fit(
            np.unique(train[usecol].unique().tolist()))#+
#                      test[usecol].unique().tolist()))

    #At the end 0 will be used for null values so we start at 1 
    train[usecol] = le.transform(train[usecol])+1
#    test[usecol]  = le.transform(test[usecol])+1

    train[usecol] = train[usecol].replace(np.nan, 0).astype('int').astype('category')
#    test[usecol]  = test[usecol].replace(np.nan, 0).astype('int').astype('category')

    joblib.dump(le, f'label_encoders/{usecol}_label_encoder.pkl')

In [ ]:
X = train.drop(['target'],axis = 1)
y = train['target']

In [ ]:
X.shape , y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=75, stratify=y)

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgb = LGBMClassifier(objective= 'binary',
        metric= 'binary_logloss',
        boosting= 'dart',
        seed= 75,
        num_leaves= 100,
        learning_rate= 0.01,
        feature_fraction= 0.20,
        bagging_freq= 10,
        bagging_fraction= 0.50,
        n_jobs= -1,
        lambda_l2= 2,
        min_data_in_leaf= 40,
        n_estimators=15000)

In [ ]:
lgb.fit(X_train, y_train, eval_metric='binary_logloss', eval_set=[(X_test, y_test)], early_stopping_rounds=100,
        verbose=150)

In [ ]:
joblib.dump(lgb, "lgb_model_with_diff.sav")

In [ ]:
del train
gc.collect()

In [ ]:
from functools import partial, reduce

models_folder = '/kaggle/working/label_encoders/'

def get_sample(path):
    test = pd.read_feather(path).set_index('customer_ID')

    test = reduce_size(test)
    
    test = test[features]
    
    gc.collect()

    for usecol in tqdm_notebook(cat_cols):
        le = joblib.load(os.path.join(models_folder,f'{usecol}_label_encoder.pkl'))
        test[usecol] = test[usecol].astype('str').apply(lambda x:x.split('.')[0])
        test[usecol] = test[usecol].map(lambda s: '<unknown>' if s not in le.classes_ else s)
        le.classes_ = np.append(le.classes_, '<unknown>')
        #At the end 0 will be used for null values so we start at 1 
        test[usecol]  = le.transform(test[usecol])+1
        test[usecol] = test[usecol].replace(np.nan, 0).astype('int').astype('category')
    return test



def get_batchs(df, batch_size, keep_features):
    n_batchs = int(len(df)/batch_size)
    cid = list(df.index)
    for i in range(n_batchs):
        idx = cid[i*batch_size:(i+1)*batch_size]
        if i == n_batchs-1:
            idx = idx + cid[(i+1)*batch_size:]
        yield df.loc[idx, keep_features]
        
def predict(sample_df, model):
    y_pred = model.predict(sample_df)
    sub = pd.Series(y_pred, index=sample_df.index, name='prediction')
#    sub.to_frame().to_csv(outfile)
    return sub

In [ ]:
n_batchs=50
sub_lg = pd.Series()
for path in glob.glob("test_*.ftr"):
    print(path)
    test = get_sample(path)
    samples_df = get_batchs(test,batch_size=n_batchs, keep_features=features)
    sample_sub_lg = pd.concat(map(partial(predict, model=lgb), tqdm_notebook(samples_df)))
    sub_lg = pd.concat([sub_lg, sample_sub_lg], ignore_index=True)
    del test
    gc.collect()
    
print(len(sub_lg))

sub_lg.to_frame().to_csv('submission_lgb_with_diff.csv')

In [ ]:
loaded_model = joblib.load('/kaggle/working/lgb_model_with_diff.sav')

In [ ]:
lgb_pred = loaded_model.predict()

In [ ]:
sample_dataset = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
output = pd.DataFrame({'customer_ID': sample_dataset.customer_ID, 'prediction': predictions})
output.to_csv('submission1.csv', index=False)